# Generic playspace

In [1]:
from catalog_manager import esm, translators
from catalog_manager.build import DFCatUpdater

/g/data/tm70/ds0092/software/mambaforge/envs/catalog-manager-dev/lib/python3.10/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
catalog_directory = "/g/data/tm70/ds0092/projects/intake_catalog/catalogs"
dfcat_name = "dfcat"

## Add existing CMIP6 esm catalog

In [3]:
from catalog_manager.translators import Cmip6MetadataTranslator

In [4]:
%%time

manager = DFCatUpdater.load_esm(
    "/g/data/hh5/public/apps/nci-intake-catalogue/esgf/cmip6/catalogue_latest.json",
    Cmip6MetadataTranslator
)
manager.add(
    name=dfcat_name,
    directory=catalog_directory
)

CPU times: user 1min 59s, sys: 2.09 s, total: 2min 1s
Wall time: 2min 1s


## Add new esm catalogs from a config file

In [5]:
import yaml

config = "/g/data/tm70/ds0092/projects/intake_catalog/config/cosima.yaml"

with open(config) as f:
    config = yaml.safe_load(f)
    
catalogs = config.get("catalogs")
builder = getattr(esm, config.get("builder"))
translator = getattr(translators, config.get("translator"))

In [6]:
%%time

for name, kwargs in catalogs.items():
    description = kwargs.pop("description")
    paths = kwargs.pop("paths")
    
    manager = DFCatUpdater.build_esm(
        name=name,
        description=description,
        builder=builder,
        translator=translator,
        paths=paths,
        builder_kwargs=kwargs,
        directory=catalog_directory,
        overwrite=True,
    )
    manager.add(
        name=dfcat_name,
        directory=catalog_directory
    )

Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_ryf9091_gadi.json
Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_iaf_omip2_cycle1.json
Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_iaf_omip2_cycle2.json
Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_iaf_omip2_cycle3.json
Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_iaf_omip2_cycle4.json
Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_iaf_omip2_cycle5.json
Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/intake_catalog/catalogs/1deg_jra55_iaf_omip2_cycle6.json
Successfully wrote ESM catalog json fi

## Use the DF catalog

In [10]:
import os
import ast
import intake

In [12]:
cat = intake.open_df_catalog(
    os.path.join(catalog_directory, f"{dfcat_name}.csv"),
    name_column="experiment",
    read_kwargs={"converters": {"variable": ast.literal_eval}},
)

In [20]:
subcat = cat.search(experiment="01deg_jra55v150_iaf_cycle1")

In [22]:
subcat.df

,experiment,frequency,model,description,realm,variable,yaml
0,01deg_jra55v150_iaf_cycle1,1day,ACCESS-OM2,0.1 degree ACCESS-OM2 global model configurati...,ice,"[time_bounds, ULON, ULAT, blkmask, aice]",sources:\n 01deg_jra55v150_iaf_cycle1:\n a...
1,01deg_jra55v150_iaf_cycle1,1mon,ACCESS-OM2,0.1 degree ACCESS-OM2 global model configurati...,ice,"[time_bounds, ULON, ULAT, blkmask, hi_m, aice_m]",sources:\n 01deg_jra55v150_iaf_cycle1:\n a...
2,01deg_jra55v150_iaf_cycle1,1day,ACCESS-OM2,0.1 degree ACCESS-OM2 global model configurati...,ocean,"[eta_t, average_T1, average_T2, average_DT, ti...",sources:\n 01deg_jra55v150_iaf_cycle1:\n a...
3,01deg_jra55v150_iaf_cycle1,1mon,ACCESS-OM2,0.1 degree ACCESS-OM2 global model configurati...,ocean,"[bmf_u, average_T1, average_T2, average_DT, ti...",sources:\n 01deg_jra55v150_iaf_cycle1:\n a...
4,01deg_jra55v150_iaf_cycle1,fx,ACCESS-OM2,0.1 degree ACCESS-OM2 global model configurati...,ocean,"[area_t, area_u, drag_coeff, dxt, dxu, dyt, dy...",sources:\n 01deg_jra55v150_iaf_cycle1:\n a...
